In [1]:
!mkdir datasets
!kaggle datasets download groleo/european-football-database

Dataset URL: https://www.kaggle.com/datasets/groleo/european-football-database
License(s): GPL-2.0




  0%|          | 0.00/1.84M [00:00<?, ?B/s]
 54%|█████▍    | 1.00M/1.84M [00:00<00:00, 1.16MB/s]
100%|██████████| 1.84M/1.84M [00:01<00:00, 2.16MB/s]
100%|██████████| 1.84M/1.84M [00:01<00:00, 1.89MB/s]


In [2]:
import zipfile

with zipfile.ZipFile('european-football-database.zip', 'r') as f:
    f.extractall('datasets/')

In [7]:
import sqlalchemy as db

engine = db.create_engine("sqlite:///datasets/european_database.sqlite")
conn = engine.connect()

In [8]:
# Extracting the metadata
metadata = db.MetaData()

# Reflect the existing database to get the tables
metadata.reflect(bind=engine)
# Print all table names to verify the available tables
print("Available tables:", metadata.tables.keys())

Available tables: dict_keys(['divisions', 'matchs'])


In [9]:
# Check if 'divisions' table exists and create a Table object for it
if 'divisions' in metadata.tables:
    division= db.Table('divisions', metadata, autoload=True, autoload_with=engine)
    # Print the metadata for the 'divisions' table
    print(repr(division))
else:
    raise Exception("Table 'divisions' does not exist in the database.")

Table('divisions', MetaData(), Column('division', TEXT(), table=<divisions>), Column('name', TEXT(), table=<divisions>), Column('country', TEXT(), table=<divisions>), schema=None)


In [10]:
# Define the table structure for 'teams'
teams = db.Table(
    'teams', metadata,
    db.Column('team_id', db.Integer, primary_key=True),
    db.Column('team_name', db.Text, nullable=False),
    db.Column('city', db.Text, nullable=False)
)

# Create the table in the database (if it does not already exist)
metadata.create_all(engine)

In [11]:
# Verify that the 'teams' table exists and print its metadata
if 'teams' in metadata.tables:
    team = metadata.tables['teams']
    print(repr(team))
else:
    raise Exception("Table 'teams' does not exist in the database.")

Table('teams', MetaData(), Column('team_id', Integer(), table=<teams>, primary_key=True, nullable=False), Column('team_name', Text(), table=<teams>, nullable=False), Column('city', Text(), table=<teams>, nullable=False), schema=None)


In [12]:
# Print all table names to verify the available tables
print("Available tables:", metadata.tables.keys())

Available tables: dict_keys(['divisions', 'matchs', 'teams'])


In [18]:
metadata.tables['teams'].columns.keys()


['team_id', 'team_name', 'city']

In [19]:
# Check if 'divisions' table exists and create a Table object for it
if 'divisions' in metadata.tables:
    division = metadata.tables['divisions']
    # Print the column names and their types
    print("Columns in 'divisions' table:")
    for column in division.columns:
        print(f"{column.name}: {column.type}")
     # Select all rows from the 'divisions' table

    # Select all rows from the 'divisions' table
    query = db.select(division)
    result_proxy = conn.execute(query)

    # Fetch all rows from the result proxy
    rows = result_proxy.fetchall()

    # Print the rows
    print("\nRows in 'divisions' table:")
    for row in rows:
        print(row)
else:
    raise Exception("Table 'divisions' does not exist in the database.")

Columns in 'divisions' table:
division: TEXT
name: TEXT
country: TEXT

Rows in 'divisions' table:
('B1', 'Division 1A', 'Belgium')
('D1', 'Bundesliga', 'Deutschland')
('D2', '2. Bundesliga', 'Deutschland')
('E0', 'Premier League', 'England')
('E1', 'EFL Championship', 'England')
('E2', 'EFL League One', 'England')
('E3', 'EFL League Two', 'England')
('EC', 'National League', 'England')
('F1', 'Ligue 1', 'France')
('F2', 'Ligue 2', 'France')
('G1', 'Superleague', 'Greece')
('I1', 'Seria A', 'Italy')
('I2', 'Seria B', 'Italy')
('N1', 'Eredivisie', 'Netherlands')
('P1', 'Liga NOS', 'Portugal')
('SC0', 'Scottish Premiership', 'Scotland')
('SC1', 'Scottish Championship', 'Scotland')
('SC2', 'Scottish League One', 'Scotland')
('SP1', 'LaLiga', 'Spain')
('SP2', 'LaLiga 2', 'Spain')
('T1', 'Süper Lig', 'Turkey')


In [20]:
query = division.select() #SELECT * FROM divisions
print(query)

SELECT divisions.division, divisions.name, divisions.country 
FROM divisions


In [21]:
exe = conn.execute(query) #executing the query
result = exe.fetchmany(5) #extracting top 5 results
print(result)

[('B1', 'Division 1A', 'Belgium'), ('D1', 'Bundesliga', 'Deutschland'), ('D2', '2. Bundesliga', 'Deutschland'), ('E0', 'Premier League', 'England'), ('E1', 'EFL Championship', 'England')]


In [26]:
engine = db.create_engine('sqlite:///datasets/datacamp.sqlite')
conn = engine.connect()
metadata = db.MetaData()

Student = db.Table('Student', metadata,
              db.Column('Id', db.Integer(),primary_key=True),
              db.Column('Name', db.String(255), nullable=False),
              db.Column('Major', db.String(255), default="Math"),
              db.Column('Pass', db.Boolean(), default=True)
              )

metadata.create_all(engine) 

In [30]:
# Insert a new row into the 'Student' table
query = db.insert(Student).values(Name='Matthew', Major='English', Pass=True)
result = conn.execute(query)
print(result)

# Fetch and print all rows from the 'Student' table to verify the insertion
query = db.select(Student)
result_proxy = conn.execute(query)
rows = result_proxy.fetchall()
print("\nRows in 'Student' table:")
for row in rows:
    print(row)


Rows in 'Student' table:
(1, 'Matthew', 'English', True)
(2, 'Matthew', 'English', True)
(3, 'Matthew', 'English', True)


In [31]:
query = db.insert(Student)
values_list = [{'Name':'Nisha', 'Major':"Science", 'Pass':False},
              {'Name':'Natasha', 'Major':"Math", 'Pass':True},
              {'Name':'Ben', 'Major':"English", 'Pass':False}]
Result = conn.execute(query, values_list)

In [33]:
output = conn.execute(db.select(Student)).fetchall()
print(output)

[(1, 'Matthew', 'English', True), (2, 'Matthew', 'English', True), (3, 'Matthew', 'English', True), (4, 'Nisha', 'Science', False), (5, 'Natasha', 'Math', True), (6, 'Ben', 'English', False)]


In [36]:
metadata.tables.keys()


dict_keys(['Student'])

In [38]:
query = db.text('SELECT * FROM "Student"')
output = conn.execute(query)
print(output.fetchall())

[(1, 'Matthew', 'English', 1), (2, 'Matthew', 'English', 1), (3, 'Matthew', 'English', 1), (4, 'Nisha', 'Science', 0), (5, 'Natasha', 'Math', 1), (6, 'Ben', 'English', 0)]


In [40]:
query = Student.select().where(Student.c.Major == 'English')
output = conn.execute(query)
print(output.fetchall())

[(1, 'Matthew', 'English', True), (2, 'Matthew', 'English', True), (3, 'Matthew', 'English', True), (6, 'Ben', 'English', False)]


In [41]:
query = Student.select().where(db.and_(Student.c.Major == 'English', Student.c.Pass != True))
output = conn.execute(query)
print(output.fetchall())

[(6, 'Ben', 'English', False)]


In [42]:
# You can copy and paste these commands to test the results on your own.

# in : Student.select().where(Student.columns.Major.in_(['English','Math']))
# and, or, not : Student.select().where(db.or_(Student.columns.Major == 'English', Student.columns.Pass = True))
# order by : Student.select().order_by(db.desc(Student.columns.Name))
# limit : Student.select().limit(3)
# sum, avg, count, min, max : db.select([db.func.sum(Student.columns.Id)])
# group by : db.select([db.func.sum(Student.columns.Id),Student.columns.Major]).group_by(Student.columns.Pass)
# distinct : db.select([Student.columns.Major.distinct()])

In [46]:
# Add a new column 'Grade' to the Student table
grade_column = db.Column('Grade', db.Float)
Student.append_column(grade_column)

# Update the table schema in the database
metadata.create_all(engine)


DuplicateColumnError: A column with name 'Grade' is already present in table 'Student'. Specify replace_existing=True to Table.append_column() to replace an existing column.

In [58]:
metadata.reflect(engine)


In [65]:
metadata.tables['Student'].c.keys()


['Id', 'Name', 'Major', 'Pass', 'Grade']

In [59]:
Student.columns.keys()


['Id', 'Name', 'Major', 'Pass', 'Grade']

In [55]:
print(Student.select())

SELECT "Student"."Id", "Student"."Name", "Student"."Major", "Student"."Pass", "Student"."Grade" 
FROM "Student"


In [56]:

# Generate random grades for existing students
import random

# Fetch all existing students
students = conn.execute(Student.select()).fetchall()

students

[(1, 'Matthew', 'English', True),
 (2, 'Matthew', 'English', True),
 (3, 'Matthew', 'English', True),
 (4, 'Nisha', 'Science', False),
 (5, 'Natasha', 'Math', True),
 (6, 'Ben', 'English', False)]

In [57]:

# Update each student with a random grade
for student in students:
    random_grade = round(random.uniform(0, 100), 2)  # Generate a random float between 0 and 100 with 2 decimal places
    update_stmt = (
        Student.update()
        .where(Student.c.Id == student.Id)
        .values(Grade=random_grade)
    )
    conn.execute(update_stmt)

conn.commit()

print("Grade column added and random grades assigned to existing students.")


OperationalError: (sqlite3.OperationalError) no such column: Grade
[SQL: UPDATE "Student" SET "Grade"=? WHERE "Student"."Id" = ?]
[parameters: (25.97, 1)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [44]:
# Print the average grade of all the students that have English major.

query = db.select(db.func.sum(Student.c.Pass)).where(Student.c.Major == 'English').group_by(Student.c.Major)
output = conn.execute(query)
print(output.fetchall())


[(True,)]
